Dynamic Table for Team

In [ ]:
USE WAREHOUSE CAT_wh;

USE DATABASE RAW3;

USE SCHEMA F1_3;

-- Creating Dynamic Table for TEAM_STATS
CREATE OR REPLACE DYNAMIC TABLE DT_TEAM_STATS 
    target_lag = 'DOWNSTREAM'
    warehouse = CAT_wh
    refresh_mode = incremental
    initialize = on_create
AS
WITH remove_duplicates AS (
    SELECT 
        ts.GRAND_PRIX,
        ts.DATE,
        ts.POINTS_WON,
        ts.YEAR,
        ts.TEAM_NAME,
        ts.TEAM_OVERALL_POS,
        ts.TOTAL_PTS,
        ROW_NUMBER() OVER (
            PARTITION BY ts.GRAND_PRIX, ts.YEAR, ts.TEAM_NAME 
            ORDER BY ts.POINTS_WON DESC
        ) AS row_num
    FROM TEAM_STATS ts
)
SELECT *
FROM remove_duplicates
WHERE row_num = 1;


-- Showing the full table
SELECT *
FROM DT_TEAM_STATS

-- Checking number of rows
SELECT COUNT(*) AS total_rows
FROM DT_TEAM_STATS;
